In [1]:
import numpy as np
import scipy as sp
from scipy import io,integrate,sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

from kpm_bin import *

from IPython.display import clear_output
np.set_printoptions(linewidth=300)
%load_ext autoreload

%autoreload 2

## TeX preamble
preamble = r"""
\usepackage[no-math]{fontspec}
\setmainfont[]{Helvetica Condensed}

\usepackage{amsmath,amssymb,amsthm}
\usepackage{mathastext}
"""

mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': preamble,
})

In [2]:
N = 20
s = 1/2
M = int(2*s+1)

d = M**N

J_t = np.zeros((N,N))
for i in range(N):
    for j in range(N):
        if np.abs(i-j)==1:
            J_t[i,j] = 1
            
J = 1
h = 12

In [3]:
H = get_hamiltonian((J/4)*J_t,(J/4)*J_t,0*J_t,h/2,s)

In [4]:
Evs = get_solvable_density_EVs(h,J,N)

In [5]:
Emin_true = Evs[0]
Emax_true = Evs[-1]

In [6]:
k = 250

np.random.seed(0)
v = np.random.randn(d)
v /= np.linalg.norm(v)

Q,(α,β) = lanczos_reorth(H,v,k,reorth=False,returnQ=True)
(α_ro,β_ro) = lanczos_reorth(H,v,k)

In [7]:
Hk = np.diag(β,1) + np.diag(β,-1) + np.diag(np.append(α,0))
e0 = np.zeros(k+1)
e0[0] = 1

Emin,Emax = Emin_true,Emax_true
μ = get_cheb_moments(Hk,e0,k,Emin,Emax)
#(γ,δ) = get_chebT_recurrence(2*k,Emin,Emax)
#μ = get_moments(Hk,e0,2*k,γ,δ)

In [8]:
μT = get_cheb_moments(H,v,k,Emin,Emax) #μT = get_moments(H,v,2*k,γ,δ)

In [9]:
orth = np.full(k,np.nan)

QTQI = Q.T@Q - np.diag(np.diag(Q.T@Q))
for i in range(k):
    orth[i] = np.max(np.abs(QTQI[:i+1,:i+1]))

In [10]:
styles = [{'color':plt.cm.magma(.85),'lw':1,'alpha':1},\
          {'color':plt.cm.magma(.5),'lw':1,'alpha':1},\
          {'color':plt.cm.magma(.12),'lw':1,'alpha':1}]

fig,axs = plt.subplots(1,2,figsize=(6,3),sharey=True)
plt.subplots_adjust(hspace=.3,bottom=.18,top=.9)

# fig,axs = plt.subplots(1,2,figsize=(12,3))
# plt.subplots_adjust(wspace=.3,bottom=.18,top=.9)

axs[1].plot(orth,**styles[0])
axs[1].plot(np.abs(α-α_ro),**styles[1])
axs[1].plot(np.abs(β-β_ro),**styles[2])

axs[1].set_yscale('log')

axs[0].plot(np.abs(μ-μT),**styles[2])


axs[0].set_xticks([0,100,200,300,400,500])
axs[1].set_xticks([0,50,100,150,200,250])

axs[1].set_xlabel('index \emph{n}')
axs[1].set_ylabel('recurrence coefficient error')

axs[0].set_xlabel('moment degree \emph{n}')
axs[0].set_ylabel('moment error')

axs[0].set_yscale('log')

axs[0].set_ylim(1e-16,1e2)
axs[1].set_ylim(1e-16,1e2)

plt.savefig('imgs/moments_err.pdf')